<a href="https://colab.research.google.com/github/Rimich666/sberautopodpiska/blob/master/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22Task3_step3_AutoML_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 0. Монтирование диска с датасетом
from google.colab import drive
drive.mount('/content/drive', force_remount=True)  # Принудительное перемонтирование

Mounted at /content/drive


In [ ]:
# 1. Установка библиотек
!pip install -U lightautoml pandas scikit-learn imbalanced-learn

  Using cached scikit_learn-1.7.1-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)


In [ ]:
# 2. Импорт модулей
import warnings
warnings.simplefilter('ignore', FutureWarning)

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    precision_score, recall_score, f1_score,
    roc_auc_score, precision_recall_curve, auc
)

from imblearn.over_sampling import RandomOverSampler

from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task

/usr/local/lib/python3.11/dist-packages/lightautoml/transformers/text.py:22: UserWarning: 'gensim' - package isn't installed
  warnings.warn("'gensim' - package isn't installed")


In [ ]:
# 3. Чтение датасета
df = pd.read_csv(
    '/content/drive/MyDrive/Проектный практикум (хакатон)/cleaned_sessions_new.csv',
    low_memory=False
)

In [ ]:
# 4) Метка «1» для целевых событий
target_events = [
    'sub_car_claim_click',
    'sub_car_claim_submit_click',
    'sub_open_dialog_click',
    'sub_custom_question_submit_click',
    'sub_call_number_click',
    'sub_callback_submit_click',
    'sub_submit_success',
    'sub_car_request_submit_click'
]
df['target_column'] = df['event_action'].isin(target_events).astype(int)

In [ ]:
# 5) Поведенческие признаки (до удаления event_action и session_id)
df['events_per_session']        = df.groupby('session_id')['event_action'].transform('count')
df['unique_events_per_session'] = df.groupby('session_id')['event_action'].transform('nunique')

In [ ]:
# 6) Временные признаки
df['visit_date'] = pd.to_datetime(df['visit_date'], format='%Y-%m-%d', errors='coerce')
df['visit_time'] = pd.to_timedelta(df['visit_time'], errors='coerce')
df['visit_dt']   = df['visit_date'] + df['visit_time']

df['visit_hour']      = df['visit_dt'].dt.hour
df['visit_dayofweek'] = df['visit_dt'].dt.dayofweek

sess_min = df.groupby('session_id')['visit_dt'].transform('min')
sess_max = df.groupby('session_id')['visit_dt'].transform('max')
df['session_duration_s'] = (sess_max - sess_min).dt.total_seconds()

In [ ]:
# 7) Удаляем колонки-утечки
df.drop([
    'visit_date', 'visit_time', 'visit_dt',
    'event_action', 'session_id'
], axis=1, inplace=True)

In [ ]:
# 8) Формируем сбалансированный подвыбор, где положительных событий в 2 раза меньше отрицательных
n_event = len(df[df['target_column'] == 1])
df_short = pd.concat([
    df[df['target_column'] == 1],
    df[df['target_column'] == 0].sample(n=2*n_event, random_state=12)
]).reset_index(drop=True)

In [ ]:
# 9) Разбиение на train/test из этого сбалансированного подвыбора
train_df, test_df = train_test_split(
    df_short,
    test_size=0.2,
    stratify=df_short['target_column'],
    random_state=42
)

In [ ]:
# 10) Настройка AutoML
automl = TabularAutoML(
    task=Task('binary'),
    timeout=600,       # 10 минут
    cpu_limit=4,       # до 4 ядер
    reader_params={'n_jobs':4}
)
roles = {'target':'target_column'}

In [ ]:
# 11) Обучение и получение OOF‑предсказаний на train
oof_pred    = automl.fit_predict(train_data=train_df, valid_data=None, roles=roles, verbose=1)
train_proba = oof_pred.data.ravel()
y_train_oof = train_df['target_column'].values

[16:28:11] Stdout logging level is INFO.


INFO:lightautoml.automl.presets.base:Stdout logging level is INFO.


[16:28:11] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer


[16:28:11] Task: binary



INFO:lightautoml.automl.presets.base:Task: binary



...

INFO:lightautoml.automl.blend:Blending: best score = 0.6856607, best weights = [0.37032697 0.15044034 0.24521966 0.16168319 0.07232989]


[16:31:47] Automl preset training completed in 215.45 seconds



INFO:lightautoml.automl.presets.base:Automl preset training completed in 215.45 seconds



[16:31:47] Model description:
Final prediction for new objects (level 0) = 
	 0.37033 * (5 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.15044 * (5 averaged models Lvl_0_Pipe_1_Mod_0_LightGBM) +
	 0.24522 * (5 averaged models Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM) +
	 0.16168 * (5 averaged models Lvl_0_Pipe_1_Mod_2_CatBoost) +
	 0.07233 * (5 averaged models Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost) 



INFO:lightautoml.automl.presets.base:Model description:
Final prediction for new objects (level 0) = 
	 0.37033 * (5 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.15044 * (5 averaged models Lvl_0_Pipe_1_Mod_0_LightGBM) +
	 0.24522 * (5 averaged models Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM) +
	 0.16168 * (5 averaged models Lvl_0_Pipe_1_Mod_2_CatBoost) +
	 0.07233 * (5 averaged models Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost) 



In [ ]:
from sklearn.metrics import roc_auc_score
import numpy as np

In [ ]:
# 12) Предсказания на тестовом наборе
test_pred  = automl.predict(test_df)
test_proba = test_pred.data.ravel()
y_test     = test_df['target_column'].values

In [ ]:
# 13) Функция для вычисления метрик
def calc_metrics(y_true, proba, thr=0.5):
    mask = ~np.isnan(proba)
    y, p = y_true[mask], proba[mask]
    preds = (p >= thr).astype(int)
    prec = precision_score(y, preds, zero_division=0)
    rec  = recall_score(y, preds, zero_division=0)
    f1   = f1_score(y, preds, zero_division=0)
    roc  = roc_auc_score(y, p)
    prc, rec_curve, _ = precision_recall_curve(y, p)
    pr_auc = auc(rec_curve, prc)
    return prec, rec, f1, roc, pr_auc

In [ ]:
# 14) Подсчёт метрик и вывод
names    = ['Precision','Recall','F1-score','ROC-AUC','PR-AUC']
mt_train = calc_metrics(y_train_oof, train_proba)
mt_test  = calc_metrics(y_test,       test_proba)

print("=== Train metrics (OOF) ===")
for n,v in zip(names, mt_train):
    print(f"{n:10s}: {v:.4f}")
print("\n=== Test metrics ===")
for n,v in zip(names, mt_test):
    print(f"{n:10s}: {v:.4f}")

=== Train metrics (OOF) ===
Precision : 0.8054
Recall    : 0.0839
F1-score  : 0.1519
ROC-AUC   : 0.6857
PR-AUC    : 0.5412

=== Test metrics ===
Precision : 0.7969
Recall    : 0.0480
F1-score  : 0.0906
ROC-AUC   : 0.6912
PR-AUC    : 0.5340
